In [1]:
import time
import LogA.script.logLibrary as lib
import LogA.script.DataLibrary as datalib
from LogA.script.zeugma import EmbeddingTransformer,TextsToSequences, Padder, ItemSelector
import tensorflow as tf
import LogA.script.OSlib as oslib
from io import StringIO
import requests
import pandas as pd
import glob
import LogA.script.Log as logMsg
import LogA.script.DataPreprocessor as dp
# Set random seeds for reproducibility
from random import seed
import numpy as np
from sklearn.preprocessing import LabelEncoder
import joblib as jb
from sklearn.pipeline import make_pipeline
#import keras.backend.tensorflow_backend as tb
# We'll use the GloVe pre-trained embeddings, using the sum of the word embeddings
# of a job title as the embedding vector
#embedding = EmbeddingTransformer('glove', aggregation='sum')
import re,os
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
seed(42)

LOG Anomaly Detection - Prediction Framework
ErrorClassification Model is loaded Sucessfully!!
PossibleCause Model is loaded Sucessfully!!
RecommendedFix Model is loaded Sucessfully!!
LogA/Config/ClassMap_hash_2hash.json


In [3]:
pridiction_file_columns = {
   "ErrorClass" : ['DateTime','LogLevel', 'Content', 'Test_Category','TestCase','LogComponent', 
                            'PredictedCategory', 'MatchingErrorMsg', 'CategoryPScore','Type'],
    "PossibleCause" : ['EventId','Level', 'Content' , 'ErrorComponent','PredictedCategory', 'MatchingErrorMsg', 'CategoryPScore','PredPossibleCause','MatchingCategory','PossiblePScore','Type'],
    "RecommendedFix" : ['EventId','Level','DateTime','Content','TestCase','PredictedCategory', 'MatchingErrorMsg', 'CategoryPScore','PredPossibleCause','MatchingCategory','PossiblePScore', 'PredictFix','MatchingCause','FixPScore','FixDisplay','Type'],
    }
input_path_ = {
    "ErrorClass" : "CAnomaly",
    "PossibleCause" : "ErrorClassification",
    "RecommendedFix" : "PossibleCause"
    }
feature_columns = {
    "ErrorClass" : ['Content'],
    "PossibleCause" : ['Content','PredictedCategory'],
    "RecommendedFix" : ['Content','PredictedCategory','PredPossibleCause']
    }

In [4]:
max_words_Error_Msg = 200
vocab_size = 2000
pipeline = oslib.getPipline(vocab_size,max_words_Error_Msg)

In [11]:
def get_prediction(ErrorMsg,siamese_lstm,x_references, train_set,features,label):
    """ Get the predicted recommended fix, and the most similar recommended fix
    in the train set. """
    x = pipeline.transform([ErrorMsg])
    # Compute similarities of the causes with all possible cause  in the train set
    similarities = siamese_lstm.predict([np.array([x[0]]*len(x_references)), x_references])
    print("similarities :",similarities)
    # print(f"similarities : {similarities}")
    most_similar_index = np.argmax(similarities)
    print("most_similar_index :",most_similar_index)
    maxValue = np.max(similarities)
    
    # The predicted category is the one of the most similar example from the train set
    prediction = train_set[label].iloc[most_similar_index]
    FixDisplay = train_set['FixDisplay'].iloc[most_similar_index]

    most_similar_example = train_set[features].iloc[most_similar_index].to_list()
    return prediction, ErrorMsg, most_similar_example, maxValue,FixDisplay

In [6]:
def predictRecommendedFix(features,label,modelType):
    # x_train, y_train, train_set = loadTrainData()
    # y_train = encode_y_target(y_train)
    # Test_log_file_path = lib.getfilelist(f"LogA/Data/Test_data/Predictions/{input_path_[modelType]}/")
    Test_log_file_path = glob.glob(r"./LogA/Data/Test_data/Predictions/PossibleCause/*.csv")

    if (len(Test_log_file_path)>=1):
        print("\nClassifying Error...\n")
        train_file = "LogA/Data/oneshot/RawTraindata.csv"
        x_train, y_train, train_set = oslib.loadTrainData(features,label,train_file)
        y_train = oslib.encode_y_target(y_train)
        trianedPairedFile = f"LogA/Data/oneshot/{modelType}/trained_paired_data.csv"
        paired_set = pd.read_csv(trianedPairedFile, sep=',')
            # Note that the preprocessing pipeline must be fit on both the right and left examples
            # simultaneously
        pipeline.fit(list(paired_set['Log_left']) + list(paired_set['Log_right']))
        x_references = pipeline.transform(x_train.agg(' & '.join,axis=1))
    cnt = 0
    destiFolder=r"LogA/Data/Test_data/Predictions/RecommendedFix/"
    if not os.path.exists(destiFolder):
        os.makedirs(destiFolder)
    for file in tqdm(Test_log_file_path):
        startTime = time.time()
        predictedResults = pd.DataFrame(columns=pridiction_file_columns[modelType])
        test_data = pd.read_csv(file, sep=',')
        cnt +=1
        predictedFile =  lib.getFileName(file, ".csv")
        dp.updatevariable(f"RecommendedFix prediction {predictedFile}",80)

        # test_set_error = test_data.query('Level!="I" and Level!="INFO" and Level!="info" and Level!="notice"')
        test_set_error = test_data
        test_set_error['ErrMsg'] = test_set_error[feature_columns[modelType]].agg(' & '.join,axis=1)
        #test_set_error['ErrMsg'] = test_set_error[feature_columns[modelType]].apply(lambda x: ' & '.join(map(str, x)), axis=1)
        # print (f" this is an anomaly file {file}")
        #test_set = test_set_error.query('prediction=="Anomaly"')
        #test_set = test_data.query('prediction=="Anomaly"')
        #test_set.append(test_set_error)
        #test_set  = DataFrame.drop_duplicates(test_set, "EventId")
        #print(f"this is my test set- check the same {test_set}")
        for row in test_set_error.itertuples():
            # print (f'i am inside the row {row}')
            # line = regex.customPreprocess(row.Content,regex_df)
            # print()
            line = row.ErrMsg
            # print("line\n",line)
            predictFix, ErrorCause, most_sim_Fix, predictFScore, fixDisplay = get_prediction(line,siamese_lstm_rFix,x_references, train_set,features,label)
            # print(f'Log Error Message: {err_data}')     
            # print(f'Predicted Error Cause: {pred}')       
            # print(f'Cause for similar Error: {most_sim}')
            predictedResults = predictedResults.append(pd.Series([row.EventId,row.Level, row.DateTime,row.Content,row.TestCase, row.PredictedCategory ,row.MatchingErrorMsg, row.CategoryPScore, row.PredPossibleCause,row.MatchingCategory, row.PossiblePScore,predictFix, most_sim_Fix, predictFScore, fixDisplay,row.Type], index=predictedResults.columns ), ignore_index=True)
            # predictedResults = predictedResults.append(pd.Series(
            #     [*[for cl in input_path_[modelType]],
            #     pred, most_sim, predictScore],
            #     index=predictedResults.columns), ignore_index=True)
        # print(file)
        eTime = time.time()-startTime
        logMsg.logUpdate(f'Parsing and Structuring data...{os.path.split(file)[1]} {os.path.getsize(file)} timetaken:{eTime}')
        
        errCount = len(predictedResults)
     
        # ErrorPredictSum = ErrorPredictSum.append(pd.Series([cnt,file,predictedFile,errCount], index=ErrorPredictSum.columns ), ignore_index=True)
        if (errCount > 0):
            # predictedResults.to_pickle(r"C:\HCL\iLA\Intel\SISW\Source\backend\Data\Oneshot\Trainold.pkl")
            # print(f"Pridict result Type is {type(predictedResults)}")
            # predictedResults = DataFrame.drop_duplicates(predictedResults)
            # predictedResults = pd.DataFrame.drop_duplicates(predictedResults)
            ErrorFixFile = predictedFile.replace(".csv_structured"," _"+str(cnt))
            predictedResults.loc[predictedResults['Content'].str.contains('In Testcase', case=False), ['PredictedCategory', 'PredPossibleCause', 'PredictFix','FixDisplay']] = 'Testcase Trigger'

            predictedResults.to_csv("./LogA/Data/Test_data/Predictions/RecommendedFix/" + predictedFile , index=False)

In [7]:
def PredictRecommendedFix():
    if siamese_lstm_rFix==None:
        raise Exception("RecommendedFix Model is Not Trained...")
    predictRecommendedFix(['MaskedContent','ErrorCategory','PossibleCause'],'RecommendedFix','RecommendedFix')


In [9]:
def initialize_model(modelType):
    model_path = f"LogA/Model/oneshot/{modelType}"
    if os.path.exists(model_path):
        global siamese_lstm_rFix
        siamese_lstm_rFix = oslib.LoadModel(model_path)
        print(f"{modelType} Model is loaded Sucessfully!!")
        return ""
    # global siamese_lstm
    print(f'{modelType} Model is not trained....')
siamese_lstm_rFix = None
initialize_model('RecommendedFix')

RecommendedFix Model is loaded Sucessfully!!


''

In [12]:
PredictRecommendedFix()


Classifying Error...



  0%|                                                                                                                        | 0/1 [00:00<?, ?it/s]

8/8 [==============================] - 0s 20ms/step
similarities : [[2.0784375e-03]
 [1.9554351e-03]
 [1.9616773e-03]
 [1.8584039e-03]
 [2.1724580e-03]
 [2.1659799e-03]
 [2.1524623e-03]
 [1.2268256e-02]
 [6.6457974e-04]
 [6.7248929e-04]
 [6.6990644e-04]
 [5.3563621e-03]
 [5.4679718e-03]
 [5.3768083e-03]
 [6.9589529e-04]
 [8.9891441e-03]
 [3.9689168e-03]
 [1.1437394e-02]
 [6.7133905e-04]
 [9.6524827e-04]
 [5.5389275e-05]
 [1.1141771e-03]
 [7.8213523e-04]
 [8.9768419e-04]
 [1.6136941e-03]
 [1.9285230e-03]
 [1.9046613e-03]
 [1.0548937e-02]
 [6.7248929e-04]
 [1.6990233e-03]
 [5.0586197e-03]
 [4.0014816e-05]
 [5.9006159e-04]
 [6.7133905e-04]
 [1.1714889e-03]
 [2.2114356e-04]
 [3.8665331e-03]
 [3.8132663e-03]
 [3.8665331e-03]
 [2.2732658e-02]
 [6.9728773e-04]
 [1.8153064e-03]
 [1.5783317e-03]
 [9.0166703e-03]
 [6.7254505e-04]
 [6.7245722e-04]
 [6.7133905e-04]
 [6.7210908e-04]
 [2.2730045e-02]
 [1.5798233e-02]
 [1.0812269e-02]
 [1.4855588e-02]
 [1.0986936e-02]
 [8.8532418e-03]
 [1.2813947e-02

8/8 [==============================] - 0s 20ms/step
similarities : [[5.0851016e-04]
 [4.3844260e-04]
 [4.2863362e-04]
 [4.4241222e-04]
 [4.6522391e-04]
 [4.6580727e-04]
 [4.7234152e-04]
 [5.5573089e-04]
 [3.5091383e-05]
 [3.5552835e-05]
 [3.5538193e-05]
 [4.3570073e-04]
 [5.4636033e-04]
 [3.8641517e-04]
 [3.7165188e-05]
 [2.7366807e-03]
 [1.2504273e-03]
 [3.2133162e-03]
 [3.5486781e-05]
 [1.2349893e-04]
 [7.1010527e-06]
 [1.8168997e-04]
 [1.1385276e-04]
 [1.3242642e-04]
 [1.1318890e-04]
 [4.2453976e-03]
 [4.1701267e-03]
 [9.7305402e-03]
 [3.5552835e-05]
 [8.8496279e-05]
 [1.5760830e-03]
 [2.1397063e-06]
 [3.1513002e-05]
 [3.5486781e-05]
 [2.2972409e-04]
 [4.3423206e-05]
 [6.4900122e-04]
 [6.3869753e-04]
 [6.4900122e-04]
 [1.3450523e-03]
 [3.7239053e-05]
 [1.7673492e-04]
 [3.8614166e-03]
 [2.7500943e-03]
 [3.5566707e-05]
 [3.5551006e-05]
 [3.5486781e-05]
 [3.5547142e-05]
 [1.3447894e-03]
 [9.3451806e-04]
 [4.4183438e-03]
 [1.2104629e-02]
 [8.8562854e-03]
 [2.6300650e-03]
 [3.1464593e-03

8/8 [==============================] - 0s 22ms/step
similarities : [[3.17775682e-02]
 [3.11046951e-02]
 [3.11230477e-02]
 [3.72053161e-02]
 [3.04737017e-02]
 [3.05795260e-02]
 [3.07806283e-02]
 [1.06378505e-02]
 [9.25894492e-05]
 [9.12636679e-05]
 [9.10343588e-05]
 [1.20498380e-03]
 [1.20752957e-03]
 [1.21803314e-03]
 [9.19583181e-05]
 [9.01344605e-03]
 [6.10071700e-03]
 [1.54213945e-03]
 [9.12479154e-05]
 [5.55951428e-03]
 [3.27103713e-04]
 [1.13142282e-02]
 [8.73760227e-03]
 [8.85564368e-03]
 [8.52154270e-02]
 [1.13700307e-03]
 [1.15752988e-03]
 [2.66830856e-03]
 [9.12636679e-05]
 [1.08753871e-02]
 [5.34226140e-03]
 [5.11347707e-06]
 [7.79732654e-05]
 [9.12479154e-05]
 [1.60783762e-03]
 [3.02830624e-04]
 [9.05593671e-03]
 [9.08493716e-03]
 [9.05593671e-03]
 [6.28931029e-03]
 [9.27755973e-05]
 [7.16904402e-02]
 [1.26247318e-03]
 [8.95422231e-03]
 [9.12325122e-05]
 [9.12628020e-05]
 [9.12479154e-05]
 [9.12459145e-05]
 [6.28841063e-03]
 [4.37039742e-03]
 [8.79681483e-03]
 [1.91212050e-0

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.57s/it]

similarities : [[3.51100625e-03]
 [4.15649638e-03]
 [4.73798532e-03]
 [4.45760740e-03]
 [4.14624438e-03]
 [4.14980296e-03]
 [3.95625550e-03]
 [4.67834901e-03]
 [9.48662683e-03]
 [9.32573713e-03]
 [9.31939762e-03]
 [4.78005037e-03]
 [4.00626659e-03]
 [5.14870230e-03]
 [9.43359174e-03]
 [3.91898118e-03]
 [6.97441678e-03]
 [1.21261822e-02]
 [9.32692830e-03]
 [2.90498044e-03]
 [1.66430065e-04]
 [3.54072754e-03]
 [2.17543519e-03]
 [2.61721411e-03]
 [1.43116256e-02]
 [6.27625384e-04]
 [6.19811879e-04]
 [1.91834883e-03]
 [9.32573713e-03]
 [3.00744846e-02]
 [2.03064503e-03]
 [5.09794976e-04]
 [7.99893495e-03]
 [9.32692830e-03]
 [2.16008537e-03]
 [4.08168853e-04]
 [5.82625996e-03]
 [6.06606994e-03]
 [5.82625996e-03]
 [9.28419828e-03]
 [9.33456793e-03]
 [8.54841340e-03]
 [5.02381241e-04]
 [3.88640002e-03]
 [9.32173524e-03]
 [9.32571944e-03]
 [9.32692830e-03]
 [9.32484306e-03]
 [9.28638503e-03]
 [6.45951834e-03]
 [1.92050892e-03]
 [2.62729847e-03]
 [2.34876992e-03]
 [4.06003697e-03]
 [1.55353295e

In [7]:
def initialize_model(modelType):
    model_path = f"LogA/Model/oneshot/{modelType}"
    if os.path.exists(model_path):
        global siamese_lstm_rFix
        siamese_lstm_rFix = oslib.LoadModel(model_path)
        print(f"{modelType} Model is loaded Sucessfully!!")
        return ""
    # global siamese_lstm
    print(f'{modelType} Model is not trained....')
siamese_lstm_rFix = None
initialize_model('RecommendedFix')

RecommendedFix Model is loaded Sucessfully!!


''